# Lecture 11: Advanced CNN

What size of filter should we use ?

## Inception

Try all possible filters and concatenate them!

Before using otherfilters, a 1x1 convolution is used First!

## Why 1x1 convolution?

Using 1x1 convolution reduces operations making the same size of feature maps!

## Implementation : Inception Module

In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable

# Training Settings
batch_size=64

# MNIST Dataset
train_dataset = datasets.MNIST(root='./mnist_data/',
                               train=True,
                               transform=transforms.ToTensor(),
                               download=True)

test_dataset = datasets.MNIST(root='./mnist_data/',
                              train=False,
                              transform=transforms.ToTensor())

# Data Loader (Input Pipeline)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)


## Inception Module

class InceptionA(nn.Module):

    def __init__(self, in_channels):
        super(InceptionA, self).__init__()
        self.branch1x1 = nn.Conv2d(in_channels, 16, kernel_size=1)

        self.branch5x5_1 = nn.Conv2d(in_channels, 16, kernel_size=1)
        self.branch5x5_2 = nn.Conv2d(16, 24, kernel_size=5, padding=2)

        self.branch3x3dbl_1 = nn.Conv2d(in_channels, 16, kernel_size=1)
        self.branch3x3dbl_2 = nn.Conv2d(16, 24, kernel_size=3, padding=1)
        self.branch3x3dbl_3 = nn.Conv2d(24, 24, kernel_size=3, padding=1)

        self.branch_pool = nn.Conv2d(in_channels, 24, kernel_size=1)

    def forward(self,x):
        branch1x1 = self.branch1x1(x)

        branch5x5 = self.branch5x5_1(x)
        branch5x5 = self.branch5x5_2(branch5x5)

        branch3x3dbl = self.branch3x3dbl_1(x)
        branch3x3dbl = self.branch3x3dbl_2(branch3x3dbl)
        branch3x3dbl = self.branch3x3dbl_3(branch3x3dbl)

        branch_pool = F.avg_pool2d(x, kernel_size=3, stride=1, padding=1)
        branch_pool = self.branch_pool(branch_pool)

        outputs = [branch1x1, branch5x5, branch3x3dbl, branch_pool]
        return torch.cat(outputs, 1)

# Model Design
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(88, 20, kernel_size=5)

        self.incept1 = InceptionA(in_channels=10)
        self.incept2 = InceptionA(in_channels=20)

        self.mp = nn.MaxPool2d(2)
        self.fc = nn.Linear(1408, 10)

    def forward(self, x):
        in_size = x.size(0)
        x = F.relu(self.mp(self.conv1(x)))
        x = self.incept1(x)
        x = F.relu(self.mp(self.conv2(x)))
        x = self.incept2(x)
        x = x.view(in_size, -1) # Flatten Tensor
        x = self.fc(x)
        return F.log_softmax(x)

model = Net()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

def train(epoch):
    model.train()
    for batch_idx, (data,target) in enumerate(train_loader):
        data, target = Variable(data), Variable(target)
        optimizer.zero_grad()
        output = model(data)   
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx%10 == 0:
             print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

def test():
    model.eval()
    test_loss = 0
    correct = 0 
    for data, target in test_loader:
        data, target = Variable(data, volatile = True), Variable(target)
        output = model(data)

        # sum up batch loss
        test_loss += F.nll_loss(output, target, size_average=False).item()
        # Get the index of the max log probability 
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()
        
    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

for epoch in range(1, 10):
    train(epoch)
    test()


/var/folders/j6/9s4ycw_d3q567rb82x3f01l40000gn/T/ipykernel_4272/656406924.py:86: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


Train Epoch: 1 [0/60000 (0%)]	Loss: 2.322916
Train Epoch: 1 [640/60000 (1%)]	Loss: 2.305119
Train Epoch: 1 [1280/60000 (2%)]	Loss: 2.291618
Train Epoch: 1 [1920/60000 (3%)]	Loss: 2.311190
Train Epoch: 1 [2560/60000 (4%)]	Loss: 2.312417
Train Epoch: 1 [3200/60000 (5%)]	Loss: 2.289750
Train Epoch: 1 [3840/60000 (6%)]	Loss: 2.294561
Train Epoch: 1 [4480/60000 (7%)]	Loss: 2.286002
Train Epoch: 1 [5120/60000 (9%)]	Loss: 2.282085
Train Epoch: 1 [5760/60000 (10%)]	Loss: 2.300316
Train Epoch: 1 [6400/60000 (11%)]	Loss: 2.286613
Train Epoch: 1 [7040/60000 (12%)]	Loss: 2.282909
Train Epoch: 1 [7680/60000 (13%)]	Loss: 2.243619
Train Epoch: 1 [8320/60000 (14%)]	Loss: 2.282791
Train Epoch: 1 [8960/60000 (15%)]	Loss: 2.238502
Train Epoch: 1 [9600/60000 (16%)]	Loss: 2.223758
Train Epoch: 1 [10240/60000 (17%)]	Loss: 2.166869
Train Epoch: 1 [10880/60000 (18%)]	Loss: 2.178447
Train Epoch: 1 [11520/60000 (19%)]	Loss: 2.038352
Train Epoch: 1 [12160/60000 (20%)]	Loss: 1.989109
Train Epoch: 1 [12800/60000 (

/var/folders/j6/9s4ycw_d3q567rb82x3f01l40000gn/T/ipykernel_4272/656406924.py:110: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  data, target = Variable(data, volatile = True), Variable(target)
/opt/homebrew/lib/python3.9/site-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))



Test set: Average loss: 0.1777, Accuracy: 9431/10000 (94%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.153150
Train Epoch: 2 [640/60000 (1%)]	Loss: 0.127153
Train Epoch: 2 [1280/60000 (2%)]	Loss: 0.074827
Train Epoch: 2 [1920/60000 (3%)]	Loss: 0.097189
Train Epoch: 2 [2560/60000 (4%)]	Loss: 0.073835
Train Epoch: 2 [3200/60000 (5%)]	Loss: 0.295483
Train Epoch: 2 [3840/60000 (6%)]	Loss: 0.286818
Train Epoch: 2 [4480/60000 (7%)]	Loss: 0.111356
Train Epoch: 2 [5120/60000 (9%)]	Loss: 0.048309
Train Epoch: 2 [5760/60000 (10%)]	Loss: 0.206512
Train Epoch: 2 [6400/60000 (11%)]	Loss: 0.294929
Train Epoch: 2 [7040/60000 (12%)]	Loss: 0.142132
Train Epoch: 2 [7680/60000 (13%)]	Loss: 0.101983
Train Epoch: 2 [8320/60000 (14%)]	Loss: 0.221479
Train Epoch: 2 [8960/60000 (15%)]	Loss: 0.153069
Train Epoch: 2 [9600/60000 (16%)]	Loss: 0.120977
Train Epoch: 2 [10240/60000 (17%)]	Loss: 0.189695
Train Epoch: 2 [10880/60000 (18%)]	Loss: 0.043171
Train Epoch: 2 [11520/60000 (19%)]	Loss: 0.064346
Train Epoch: 2 [12

More layers give more accurate results.
Can we just go deeper, keep stacking layers?

No!

## Problems with stacking layers
- Vanishing gradients problem
- Back propagation kind of gives up...
- Degradation problem - with increased network depth accuracy gets saturated and then rapidly degrades.

## Solution : Deep Residual learning

Add the input to the output of each layer to help back propagation.

